In [3]:
import sys
from pathlib import Path

# Add project root to sys.path so we can import from utils
PROJECT_ROOT = Path.cwd()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))
    
from utils.process_images import process_images
from utils.label_images import label_images_interactively
from utils.analyze_segment import process as analyze_segments
from utils.create_cards import create_cards_interactive
from utils.generate_printable_cards import generate_printable_cards
from utils.generate_game_pdf import create_presentation

Project root added to sys.path: /Users/dangod/Documents/GitHub/satellite-bingo


In [ ]:
# --- Configuration ---

# Directories
RAW_IMAGES_DIR = PROJECT_ROOT / "sat_images"
PROCESSED_IMAGES_DIR = PROJECT_ROOT / "converted_images"
MASK_DIR = PROJECT_ROOT / "mask"
PRINTABLE_CARDS_DIR = PROJECT_ROOT / "printable_cards"

# Files
MAPPING_FILE = PROJECT_ROOT / "image_feature_map.json"
STATS_FILE = MASK_DIR / "segmentation_stats.csv"
CARDS_FILE = PROJECT_ROOT / "bingo_cards.json"
GAME_PDF = PROJECT_ROOT / "bingo_game_presentation.pdf"

# Ensure directories exist
PROCESSED_IMAGES_DIR.mkdir(exist_ok=True)
MASK_DIR.mkdir(exist_ok=True)
PRINTABLE_CARDS_DIR.mkdir(exist_ok=True)

## Step 1: Process Images
Resize and convert images to a standard format.

In [3]:
process_images(RAW_IMAGES_DIR, PROCESSED_IMAGES_DIR, scale=1.0)

Saved: /Users/dangod/Documents/GitHub/satellite-bingo/converted_sat_images/image_1.jpg
Saved: /Users/dangod/Documents/GitHub/satellite-bingo/converted_sat_images/image_2.jpg
Saved: /Users/dangod/Documents/GitHub/satellite-bingo/converted_sat_images/image_3.jpg
Saved: /Users/dangod/Documents/GitHub/satellite-bingo/converted_sat_images/image_4.jpg
Saved: /Users/dangod/Documents/GitHub/satellite-bingo/converted_sat_images/image_5.jpg
Saved: /Users/dangod/Documents/GitHub/satellite-bingo/converted_sat_images/image_6.jpg
Saved: /Users/dangod/Documents/GitHub/satellite-bingo/converted_sat_images/image_7.jpg
Saved: /Users/dangod/Documents/GitHub/satellite-bingo/converted_sat_images/image_8.jpg
Saved: /Users/dangod/Documents/GitHub/satellite-bingo/converted_sat_images/image_9.jpg
Saved: /Users/dangod/Documents/GitHub/satellite-bingo/converted_sat_images/image_10.jpg
Saved: /Users/dangod/Documents/GitHub/satellite-bingo/converted_sat_images/image_11.jpg
Saved: /Users/dangod/Documents/GitHub/sat

## Step 2: Label Images
Use the interactive widget to label which features appear in each image.

In [5]:
FEATURES_LIST = [
    "container ship",
    "red car",
    "yellow car",
    "green car",
    "football",
    "football field",
    "baseball court",
    "tennis court",
    "space shuttle",
    "island",
    "ferry",
    "small boat",
    "golf bunker",
    "pool",
    "blue plane",
    "plane",
    "batman logo",
    "wave",
    "coca cola logo",
    "solar panel array",
    "empty parking lot",
]

mapping = label_images_interactively(folder=PROCESSED_IMAGES_DIR,features=FEATURES_LIST, out_file=MAPPING_FILE)

## Step 3: Analyze Segments (SAM)
Run the Segment Anything Model to detect objects and generate statistics.
**Note:** This step may take some time depending on your hardware (GPU recommended).

In [ ]:
# You can set device='cpu' if you don't have a GPU, but it will be slower.
# If you need to access gated models, provide your Hugging Face token here.
HF_TOKEN = None # "your_token_here"

df_stats = analyze_segments(
    mapping_path=MAPPING_FILE, 
    out_dir=MASK_DIR, 
    device='gpu', 
    resume=True,
    hf_token=HF_TOKEN
)

Using Apple Silicon GPU (MPS)
Initializing SamGeo3 device: mps
Using mps device and transformers backend


Loading weights:   0%|          | 0/1468 [00:00<?, ?it/s]

Processing image: /Users/dangod/Documents/GitHub/satellite-bingo/converted_sat_images/image_1.jpg, features: ['red car', 'yellow car']
  Skipping red car (already exists)
  Generating masks for feature: yellow car
No objects found. Please try a different prompt.
  save_masks skipped: No masks found. Please run generate_masks() first.
Processing image: /Users/dangod/Documents/GitHub/satellite-bingo/converted_sat_images/image_10.jpg, features: ['red car', 'pool', 'wave']
  Skipping red car (already exists)
  Skipping pool (already exists)
  Skipping wave (already exists)
Skipping /Users/dangod/Documents/GitHub/satellite-bingo/converted_sat_images/image_11.jpg: no features selected.
Processing image: /Users/dangod/Documents/GitHub/satellite-bingo/converted_sat_images/image_12.jpg, features: ['plane', 'batman logo']
  Skipping plane (already exists)
  Skipping batman logo (already exists)
Processing image: /Users/dangod/Documents/GitHub/satellite-bingo/converted_sat_images/image_13.jpg, fe

## Step 4: Create Bingo Cards
Generate fair Bingo cards based on the statistics. You can select which types of events (Contains, More than N, Exactly N) to include.

In [ ]:
create_cards_interactive(
    stats_path=STATS_FILE, 
    output_path=CARDS_FILE,
    num_cards=50,
    card_size=10,
    tolerance=1,
    target_difficulty=None # Set to a number (e.g. 45) to force a set difficulty level, or None to auto-calculate
)

Accordion(children=(VBox(children=(Label(value='Feature: batman logo'), Checkbox(value=True, description='Cont…

Button(description='Confirm Selections & Generate Cards', style=ButtonStyle())

Output()

## Step 5: Generate Printable Cards
Create a PDF or images of the generated cards for printing.

In [8]:
generate_printable_cards(json_path=CARDS_FILE, output_dir=PRINTABLE_CARDS_DIR)

Generating 9 pages into /Users/dangod/Documents/GitHub/satellite-bingo/printable_cards ...
Saved /Users/dangod/Documents/GitHub/satellite-bingo/printable_cards/bingo_cards_page_1.jpg
Saved /Users/dangod/Documents/GitHub/satellite-bingo/printable_cards/bingo_cards_page_2.jpg
Saved /Users/dangod/Documents/GitHub/satellite-bingo/printable_cards/bingo_cards_page_3.jpg
Saved /Users/dangod/Documents/GitHub/satellite-bingo/printable_cards/bingo_cards_page_4.jpg
Saved /Users/dangod/Documents/GitHub/satellite-bingo/printable_cards/bingo_cards_page_5.jpg
Saved /Users/dangod/Documents/GitHub/satellite-bingo/printable_cards/bingo_cards_page_6.jpg
Saved /Users/dangod/Documents/GitHub/satellite-bingo/printable_cards/bingo_cards_page_7.jpg
Saved /Users/dangod/Documents/GitHub/satellite-bingo/printable_cards/bingo_cards_page_8.jpg
Saved /Users/dangod/Documents/GitHub/satellite-bingo/printable_cards/bingo_cards_page_9.jpg
Done! Check the 'printable_cards' folder.


## Step 6: Generate Game Presentation
Create the final PDF presentation to play the game.

In [12]:
create_presentation(stats_path=STATS_FILE, cards_path=CARDS_FILE, images_dir=PROCESSED_IMAGES_DIR, output_pdf=GAME_PDF)

Generating Bingo Presentation...
Shuffled 35 images.
The winner won on turn 17
Second place won on turn 20
Third place won on turn 20
Presentation saved to /Users/dangod/Documents/GitHub/satellite-bingo/bingo_game_presentation.pdf
